# 2주차 — 기상·기후 데이터 전처리


## 학습 목표
- 결측/이상치 처리, 파생지표(누적강우/엽면수분) 생성
- 바이어스 보정 개념 훑어보기(실습은 관측 일자료 중심)

In [ ]:
import sys
import pandas as pd, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import font_manager

# ✅ 1) 한글 폰트 설정 (없으면 영어로 fallback)
candidates = [
    "Malgun Gothic",            # Windows 기본
    "NanumGothic", "Nanum Gothic",
    "Noto Sans CJK KR", "Noto Sans KR",
    "AppleGothic",
    "UnDotum", "Batang", "Gulim"
]
installed = {f.name for f in font_manager.fontManager.ttflist}
choice = next((c for c in candidates if c in installed), None)

if choice:
    mpl.rcParams["font.family"] = choice
    print("✔ Matplotlib 글꼴:", choice)
else:
    print("⚠ 한글 폰트 미탑재 → 제목/레이블은 영어로 대체 권장")

mpl.rcParams["axes.unicode_minus"] = False  # 마이너스 기호 깨짐 방지

# ✅ 2) utils 경로 추가 & 함수 임포트
UTILS_DIR = Path("..") / "utils"
if not UTILS_DIR.exists():
    # 노트북 실행 위치가 다를 수 있으니 대안 경로도 탐색
    alt = Path.cwd().resolve()
    tried = [UTILS_DIR, alt / "utils", alt.parent / "utils"]
    print("utils 탐색:", [str(p) for p in tried])
    for p in tried:
        if (p / "utils.py").exists():
            UTILS_DIR = p
            break
sys.path.append(str(UTILS_DIR))
from utils import rolling_leaf_wetness_hours

weather = pd.read_csv(str(Path("..")/"data"/"sample_weather_2022.csv"))
weather['date'] = pd.to_datetime(weather['date'])

# 파생 지표
weather['prcp_3d_sum']   = weather['prcp_mm'].rolling(3, min_periods=1).sum()
weather['leaf_wet_3d_sum'] = rolling_leaf_wetness_hours(weather['leaf_wetness_hours'], window=3)

# ── 저장 폴더 만들기 ────────────────────────────────────
FIG_DIR = Path("figs"); FIG_DIR.mkdir(exist_ok=True)

# ── 그림 1: 3일 누적 강우 ───────────────────────────────
plt.figure(figsize=(10,4))
plt.plot(weather['date'], weather['prcp_3d_sum'])
plt.title("3일 누적 강우 (prcp_3d_sum)")
plt.xlabel("날짜"); plt.ylabel("mm")
plt.tight_layout()
out1 = FIG_DIR / "02_prcp3d_sum.png"
plt.savefig(out1, dpi=150)
plt.show()
print("Saved:", out1.resolve())

# ── 그림 2: 3일 누적 엽면수분 ───────────────────────────
plt.figure(figsize=(10,4))
plt.plot(weather['date'], weather['leaf_wet_3d_sum'])
plt.title("3일 누적 엽면수분 (leaf_wet_3d_sum)")
plt.xlabel("날짜"); plt.ylabel("hours")
plt.tight_layout()
out2 = FIG_DIR / "02_leafwet3d_sum.png"
plt.savefig(out2, dpi=150)
plt.show()
print("Saved:", out2.resolve())



In [ ]:
# (선택) 비교용: 3일 누적 강우 vs 엽면수분 (한 그래프, 보조축)
FIG_DIR = Path("figs"); FIG_DIR.mkdir(exist_ok=True)

fig, ax1 = plt.subplots(figsize=(10,4))

# 왼쪽 y축: 강우 (파란색)
line1, = ax1.plot(weather['date'], weather['prcp_3d_sum'],
                   color='tab:blue', label='3일 누적 강우 (mm)')
ax1.set_xlabel("날짜")
ax1.set_ylabel("강우 (mm)", color='tab:blue')
ax1.tick_params(axis='y', colors='tab:blue')

# 오른쪽 y축: 엽면수분 (주황색)
ax2 = ax1.twinx()
line2, = ax2.plot(weather['date'], weather['leaf_wet_3d_sum'],
                   color='tab:orange', label='3일 누적 엽면수분 (h)')
ax2.set_ylabel("엽면수분 (hours)", color='tab:orange')
ax2.tick_params(axis='y', colors='tab:orange')

plt.title("3일 누적 강우 vs 엽면수분")
plt.tight_layout()

# 범례(두 축 라인 합치기)
lines = [line1, line2]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

# 저장
out3 = FIG_DIR / "02_prcp_vs_leafwet_3d_colored.png"
plt.savefig(out3, dpi=150)
plt.show()
print("Saved:", out3.resolve())